<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/geodesics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geodesics Experiment: Solar System Tests

This experiment tests our computational-capacity model against classic solar system observations: light bending, Shapiro delay, and Mercury perihelion precession.

## Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone and setup (idempotent)
import os, sys, subprocess, shutil, pathlib
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL

%cd $REPO_NAME

# Install package
if (pathlib.Path("pyproject.toml")).exists():
    !pip install -e .

# Set random seed for reproducibility
import numpy as np, random
np.random.seed(42)
random.seed(42)

## Run Geodesics Experiment

In [ ]:
from experiments.run_geodesics import main
main()

## Display Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display generated plots
output_dir = Path("experiments/out/geodesics")

print("Light Bending vs Impact Parameter:")
display(Image(str(output_dir / "light_bending.png")))

print("\nShapiro Delay vs Impact Parameter:")
display(Image(str(output_dir / "shapiro_delay.png")))

print("\nMercury Perihelion Precession:")
display(Image(str(output_dir / "mercury_precession.png")))

print("\nSolar System Tests Summary:")
display(Image(str(output_dir / "solar_system_tests.png")))

# Display numerical results
print("\nNumerical Results:")
with open(output_dir / "results.txt", 'r') as f:
    print(f.read())

## Optional: Custom Solar System Parameters

In [ ]:
# Explore different planetary systems
import numpy as np
import matplotlib.pyplot as plt
from tacc import geodesics

# Solar system parameters
GM_sun = 1.327e20  # m^3/s^2
c = 2.998e8  # m/s

# Test with different planetary parameters
planets = {
    'Mercury': {'a': 0.387, 'e': 0.206},
    'Venus': {'a': 0.723, 'e': 0.007},
    'Earth': {'a': 1.000, 'e': 0.017},
    'Mars': {'a': 1.524, 'e': 0.094}
}

gamma_values = [0.5, 1.0, 1.5, 2.0]
beta = 1.0  # Always 1 in our model

plt.figure(figsize=(12, 8))

for i, (planet_name, params) in enumerate(planets.items()):
    plt.subplot(2, 2, i+1)
    
    precessions = []
    for gamma in gamma_values:
        precession = geodesics.perihelion_precession(params['a'], params['e'], beta, gamma, GM_sun, c)
        # Convert to arcseconds per century
        if planet_name == 'Mercury':
            period_days = 88
        elif planet_name == 'Venus':
            period_days = 225
        elif planet_name == 'Earth':
            period_days = 365
        elif planet_name == 'Mars':
            period_days = 687
        
        precession_arcsec = precession * 206265 * 100 / (period_days * 365.25 * 24 * 3600) * (100 * 365.25 * 24 * 3600)
        precessions.append(precession_arcsec)
    
    plt.plot(gamma_values, precessions, 'o-', linewidth=2, markersize=6)
    plt.xlabel('γ (PPN Parameter)')
    plt.ylabel('Precession (arcsec/century)')
    plt.title(f'{planet_name} (a={params["a"]} AU, e={params["e"]})')
    plt.grid(True, alpha=0.3)
    
    # Add observed value for Mercury
    if planet_name == 'Mercury':
        plt.axhline(y=43.1, color='r', linestyle='--', alpha=0.7, label='Observed')
        plt.legend()

plt.suptitle('Perihelion Precession for Different Planets')
plt.tight_layout()
plt.show()

print("Key Observations:")
print("- Mercury shows the largest precession effect due to its proximity to the Sun")
print("- The effect scales with orbital parameters and proximity to the gravitational source")
print("- γ=1 (Einstein's GR) best matches the observed Mercury precession anomaly")

## Custom Light Bending Analysis

In [ ]:
# Detailed light bending analysis
import numpy as np
import matplotlib.pyplot as plt
from tacc import geodesics

# Solar parameters
GM_sun = 1.327e20  # m^3/s^2
R_sun = 6.96e8  # solar radius in meters
c = 2.998e8  # m/s

# Famous eclipse measurements
eclipse_data = {
    '1919 Sobral': {'measured': 1.98, 'error': 0.12},  # arcseconds
    '1919 Principe': {'measured': 1.61, 'error': 0.30},
    'Modern average': {'measured': 1.750, 'error': 0.009}
}

# Calculate predictions for different gamma values at solar limb
gamma_values = np.linspace(0.5, 2.0, 100)
predictions = []

for gamma in gamma_values:
    # At solar limb: b ≈ R_sun, so b/rs ≈ 1
    deflection = geodesics.null_geodesic_deflection(1.0, gamma, GM_sun, c)
    deflection_arcsec = deflection * 206265
    predictions.append(deflection_arcsec)

plt.figure(figsize=(10, 6))
plt.plot(gamma_values, predictions, 'b-', linewidth=2, label='Our Model')

# Plot historical measurements
colors = ['red', 'green', 'purple']
for i, (name, data) in enumerate(eclipse_data.items()):
    plt.axhline(y=data['measured'], color=colors[i], linestyle='--', 
               label=f'{name}: {data["measured"]}±{data["error"]}"')
    plt.fill_between(gamma_values, 
                    data['measured'] - data['error'],
                    data['measured'] + data['error'],
                    color=colors[i], alpha=0.1)

# Mark Einstein's prediction (gamma=1)
einstein_pred = geodesics.null_geodesic_deflection(1.0, 1.0, GM_sun, c) * 206265
plt.axvline(x=1.0, color='black', linestyle=':', alpha=0.7, label='Einstein (γ=1)')
plt.scatter([1.0], [einstein_pred], color='black', s=100, zorder=10)

plt.xlabel('γ (PPN Parameter)')
plt.ylabel('Light Deflection at Solar Limb (arcseconds)')
plt.title('Light Bending: Model vs Historical Measurements')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([1.0, 2.5])

plt.tight_layout()
plt.show()

print(f"Einstein's prediction (γ=1): {einstein_pred:.3f} arcseconds")
print("Historical context:")
print("- 1919 eclipse expeditions confirmed Einstein's prediction")
print("- Modern measurements agree with GR to high precision")
print("- Our model reduces to GR when γ=1 (κ=2)")

## Troubleshooting

**Common Issues:**
- Large numbers in calculations - results are in SI units
- Import errors indicate repository setup problems

**Physical Insights:**
- Light bending scales as (1+γ) - our model allows tuning via γ
- Shapiro delay also depends on (1+γ) 
- Mercury precession scales as (2-β+2γ), with β=1 in our model
- All effects reduce to Einstein's GR when γ=1

**Key Results:**
- γ=1 (κ=2) gives exact agreement with GR and observations
- The computational capacity parameter κ controls deviations from GR
- Solar system tests constrain κ ≈ 2 for consistency with observations